# 🧠 Choix du Modèle et Justifications

## 1. 🎯 Objectif du système

L'objectif de ce projet est de concevoir un **système de recommandation de contenu** capable de générer des suggestions personnalisées de vidéos à partir des **profils utilisateurs** et des **caractéristiques des vidéos**. Une attention particulière est portée à la gestion des **cold users** (utilisateurs sans historique d'interaction).

---

## 2. ⚙️ Type de modèle choisi : **Content-Based Filtering**

J'ai opté pour une approche **content-based** car elle présente plusieurs avantages dans notre contexte :

- **Indépendance par rapport aux autres utilisateurs** : utile lorsque l’historique est partiel ou inexistant.
- **Exploitation directe des caractéristiques vidéos** : via des vecteurs de contenu normalisés.
- **Simplicité de déploiement** : le modèle peut fonctionner avec de nouvelles vidéos sans nécessiter de retrain.

---

## 3. 👥 Gestion des cold users via **Réseau Social**

Un des points critiques est la recommandation pour les **utilisateurs sans profil (cold start)**. Pour cela, j'ai introduit une **propagation de profils via le graphe social**, selon cette logique :

- Si un utilisateur n’a pas de données d’interactions, on récupère les profils de ses **amis directs** (et indirects jusqu'à une profondeur `d`).
- Ces profils sont **pondérés** par un facteur de décroissance exponentielle `1 / coef_decay^depth`, reflétant l’influence décroissante des amis lointains.
- Cela permet de **reconstruire un profil estimé cohérent** à partir du comportement de la communauté.

---

In [16]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import ast
import os
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import time
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('stopwords')
nltk.download('wordnet')

def cosine_similarity_vec(a, B):
    """
    Calcule la similarité cosinus entre un vecteur a (1D) et une matrice B (2D),
    retourne un vecteur des similarités (a vs chaque ligne de B).
    Utilise scikit-learn pour la cohérence.
    """
    # Reshape a en matrice 2D (nécessaire pour scikit-learn)
    a_reshaped = a.reshape(1, -1)
    # Utiliser la fonction scikit-learn
    sim = cosine_similarity(a_reshaped, B).flatten()
    return sim


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\cypri\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\cypri\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Script : `run_optimized_recommender`

Ce script implémente un système de recommandation **content-based** (basé sur le contenu des vidéos) avec gestion des utilisateurs froids (**cold users**) à l’aide d’un **réseau social**. Il permet de générer des profils utilisateurs optimisés et de les sauvegarder pour un usage ultérieur.

---

## Objectif

Créer des **profils utilisateurs** à partir de leurs interactions vidéo (watch ratio) et, pour ceux sans interactions (cold users), estimer leur profil à partir de ceux de leurs amis (propagation dans le réseau social).

## Étapes principales

### 1. Chargement des données
- Lecture des vidéos et du réseau social.
- Chargement en batch de la `big_matrix` contenant les interactions utilisateurs.

### 2. Traitement des vidéos
- Chaque vidéo possède un vecteur de caractéristiques.
- Création d’une matrice `item_features`.

### 3. Création des profils utilisateurs
- Pour chaque utilisateur, on construit un vecteur de profil basé sur les vidéos regardées pondérées par leur **watch ratio**
- Si `watch_ratio` > 0, le profil est pondéré plus fortement (avec un carré : `** 2`).

### 4. Gestion des **cold users**
- Pour les utilisateurs sans interaction :
  - Propagation de l’information via leurs amis directs et indirects (jusqu’à `max_depth`).
  - Chaque niveau a une **influence décroissante** (par `coef_decay`).
  - Si aucun ami utile, on leur affecte un **profil moyen global**.

### 5. Sauvegarde du modèle
- Le modèle est sauvé via `pickle` et contient :
  - Les caractéristiques des vidéos.
  - Les profils utilisateurs (froids + actifs).
  - Les mappings `video_id ↔ index`.


In [ ]:
import pandas as pd
import numpy as np
import pickle
import time
import os
import gc
import ast  # pour convertir les listes string en listes Python
from collections import deque

def run_optimized_recommender(big_matrix_path, item_categories_path, social_network_path, 
                             save_path='models/content_recommender.pkl', max_users=None, batch_size=1000,
                             max_depth=3, coef_decay=2.0):
    """
    Exécute le système de recommandation content-based optimisé avec intégration des profils cold users via réseau social.
    
    Args:
        max_depth: profondeur max pour propagation amis (1 = amis directs)
        coef_decay: facteur de décroissance des coefficients (exponentiel)
    """
    os.makedirs(os.path.dirname(save_path), exist_ok=True)
    
    print("1. Chargement des métadonnées...")
    item_categories = pd.read_csv(item_categories_path)
    social_network = pd.read_csv(social_network_path)
    
    print("2. Préparation des caractéristiques de contenu...")
    max_len = 100  # taille max des vecteurs
    def pad_vector(v):
        v = ast.literal_eval(v)
        if len(v) > max_len:
            return v[:max_len]
        else:
            return v + [0]*(max_len - len(v))

    item_features = item_categories['feat'].apply(pad_vector).tolist()
    item_features = np.array(item_features)

    # Mappings video_id <-> index
    video_indices = {vid: idx for idx, vid in enumerate(item_categories['video_id'])}
    video_id_to_index = {idx: vid for vid, idx in video_indices.items()}
    
    print("3. Chargement et traitement des interactions (big_matrix) par lots...")
    
    user_profiles = {}
    users_processed = set()
    total_processed = 0
    start_time = time.time()

    for chunk_idx, chunk in enumerate(pd.read_csv(big_matrix_path, chunksize=batch_size)):
        
        if 'watch_ratio' not in chunk.columns:
            chunk['watch_ratio'] = (chunk['play_duration'] / chunk['video_duration']).clip(0,1)
        
        for user_id, user_data in chunk.groupby('user_id'):
            if user_id in users_processed:
                continue
            
            user_profile = np.zeros(item_features.shape[1])
            video_count = 0
            
            for _, row in user_data.iterrows():
                vid = row['video_id']
                watch_ratio = row['watch_ratio']
                if vid in video_indices:
                    vid_idx = video_indices[vid]
                    video_feat = item_features[vid_idx]

                    user_profile += video_feat * (watch_ratio ** 2)
                    video_count += 1
            
            if video_count > 0:
                user_profile /= video_count
                user_profiles[user_id] = user_profile
            else:
                continue
            
            users_processed.add(user_id)
            total_processed += 1
            
            if max_users and total_processed >= max_users:
                break
        
        if max_users and total_processed >= max_users:
            break
        
        gc.collect()
    
    print("4. Construction des profils pour cold users à partir du réseau social...")

    # Convertir friend_list de string à liste Python
    social_network['friend_list'] = social_network['friend_list'].apply(ast.literal_eval)
    social_dict = dict(zip(social_network['user_id'], social_network['friend_list']))

    # Profil moyen global pour fallback
    all_profiles = np.array(list(user_profiles.values()))
    mean_profile = np.mean(all_profiles, axis=0)

    # Tous les utilisateurs présents dans big_matrix
    # Tous les utilisateurs présents dans le dataset de test (small_matrix)
    all_user_ids = set(pd.read_csv('KuaiRec 2.0/data/small_matrix.csv')['user_id'].unique())
    cold_users = all_user_ids - set(user_profiles.keys())


    print(f"Nombre de cold users détectés: {len(cold_users)}")

    # Calcul des coefficients de pondération décroissants
    coefficients = [1 / (coef_decay ** i) for i in range(max_depth)]

    from collections import deque

    def get_influenced_profile(user_id):
        visited = set([user_id])
        queue = deque([(user_id, 0)])  # (user_id, profondeur)
        agg_profile = None
        total_weight = 0

        while queue:
            current_user, depth = queue.popleft()
            if depth > 0 and current_user in user_profiles:
                weight = coefficients[depth - 1]
                if agg_profile is None:
                    agg_profile = weight * user_profiles[current_user]
                else:
                    agg_profile += weight * user_profiles[current_user]
                total_weight += weight
            
            if depth < max_depth:
                friends = social_dict.get(current_user, [])
                for f in friends:
                    if f not in visited:
                        visited.add(f)
                        queue.append((f, depth + 1))
        if agg_profile is not None:
            return agg_profile / total_weight
        else:
            return None

    # Estimer profil cold users par propagation dans réseau social
    for cold_user in cold_users:
        profile = get_influenced_profile(cold_user)
        if profile is not None:
            user_profiles[cold_user] = profile
        else:
            user_profiles[cold_user] = mean_profile

    print(f"Profils pour cold users créés avec propagation à profondeur {max_depth}.")

    print(f"5. Sauvegarde des profils de {len(user_profiles)} utilisateurs...")
    
    model_data = {
        'item_features': item_features,
        'user_profiles': user_profiles,
        'video_indices': video_indices,
        'video_id_to_index': video_id_to_index
    }
    
    with open(save_path, 'wb') as f:
        pickle.dump(model_data, f)
    
    print(f"Modèle sauvegardé dans {save_path}")
    print(f"Temps total d'exécution: {(time.time() - start_time)/60:.2f} minutes")
    
    return model_data


run_optimized_recommender(
    big_matrix_path='KuaiRec 2.0/data/big_matrix.csv',
    item_categories_path='KuaiRec 2.0/data/item_categories.csv',
    social_network_path='KuaiRec 2.0/data/social_network.csv',
    save_path='models/content_recommender.pkl',
    max_users=10000,
    batch_size=500,
    max_depth=5,
    coef_decay=5
)

print("Recommandation optimisée terminée.")


1. Chargement des métadonnées...
2. Préparation des caractéristiques de contenu...
3. Chargement et traitement des interactions (big_matrix) par lots...


# Fonctions d’évaluation des recommandations

Ces trois fonctions permettent de mesurer la qualité des recommandations faites par un système.

---

## 🔹 1. `calculate_precision_at_k(...)`

### Objectif :
Mesurer **la proportion d’éléments recommandés parmi les `k` premiers qui sont pertinents**.

---

## 🔸 2. `calculate_recall_at_k(...)`

### Objectif :
Mesurer **la proportion de vidéos pertinentes qui ont été retrouvées dans les `k` recommandations**.

---

## 🔷 3. `calculate_ndcg_at_k(...)`

### Objectif :
Mesurer **la qualité de l’ordre des recommandations**, en tenant compte de leur pertinence.



In [14]:
def calculate_precision_at_k(recommended_items, relevant_items, k=10):
    """
    Calcule la précision@k (proportion d'items recommandés qui sont pertinents)
    """
    if len(recommended_items) == 0:
        return 0.0
    
    # Limiter à k recommandations
    recommended_k = recommended_items[:k]
    # Compter les items pertinents parmi les recommandations
    relevant_count = len(set(recommended_k) & set(relevant_items))
    
    return relevant_count / min(k, len(recommended_k))

def calculate_recall_at_k(recommended_items, relevant_items, k=10):
    """
    Calcule le rappel@k (proportion d'items pertinents qui ont été recommandés)
    """
    if len(relevant_items) == 0:
        return 0.0
    
    # Limiter à k recommandations
    recommended_k = recommended_items[:k]
    # Compter les items pertinents parmi les recommandations
    relevant_count = len(set(recommended_k) & set(relevant_items))
    
    return relevant_count / len(relevant_items)

def calculate_ndcg_at_k(recommended_items, relevant_items_with_ratings, k=10):
    """
    Calcule le NDCG@k (Normalized Discounted Cumulative Gain)
    
    Args:
        recommended_items: Liste des items recommandés
        relevant_items_with_ratings: Dictionnaire {item_id: rating}
        k: Nombre de recommandations à considérer
    """
    if len(recommended_items) == 0 or len(relevant_items_with_ratings) == 0:
        return 0.0
    
    # Limiter à k recommandations
    recommended_k = recommended_items[:k]
    
    # Calculer le DCG (Discounted Cumulative Gain)
    dcg = 0.0
    for i, item_id in enumerate(recommended_k):
        if item_id in relevant_items_with_ratings:
            
            dcg += (2 ** relevant_items_with_ratings[item_id] - 1) / np.log2(i + 2)
    
    # Calculer le DCG idéal
    # Trier les ratings par ordre décroissant
    sorted_ratings = sorted(relevant_items_with_ratings.values(), reverse=True)
    idcg = 0.0
    for i, rating in enumerate(sorted_ratings[:k]):
        idcg += (2 ** rating - 1) / np.log2(i + 2)
    
    # Éviter la division par zéro
    if idcg == 0:
        return 0.0
    
    return dcg / idcg


## Objectif

- Charger un modèle de recommandation à partir d’un fichier.
- Calculer la **similarité** entre le profil de l'utilisateur et les vidéos disponibles.
- Retourner les vidéos les plus proches du profil utilisateur (recommandées).

## Étapes détaillées

1. **Chargement du modèle** :
   - Le fichier contient :
     - `item_features` : vecteurs décrivant les vidéos.
     - `user_profiles` : vecteurs décrivant les utilisateurs.
     - `video_id_to_index` : correspondance entre les vidéos et leurs indices.

2. **Vérification** : si l’utilisateur n’a pas de profil, la fonction retourne une liste vide (`[]`).

3. **Calcul de similarité** :
   - Utilise une fonction externe `cosine_similarity_vec(user_profile, item_features)` pour comparer le profil utilisateur à tous les profils vidéo.

4. **Tri des résultats** :
   - Trie les vidéos par similarité décroissante (les plus proches en premier).

5. **Construction de la liste finale** :
   - Récupère les identifiants des vidéos les plus proches.
   - S’arrête une fois `n_recommendations` vidéos sélectionnées.

In [12]:
def load_and_recommend(model_path, user_id, n_recommendations=10):
    """
    Charge un modèle et génère des recommandations pour un utilisateur
    """
    with open(model_path, 'rb') as f:
        model_data = pickle.load(f)
    
    item_features = model_data['item_features']
    user_profiles = model_data['user_profiles']
    video_indices = model_data['video_indices']
    video_id_to_index = model_data['video_id_to_index']
    
    if user_id not in user_profiles:
        return []
    
    user_profile = user_profiles[user_id]
    
    # Calculer la similarité
    user_profile = user_profiles[user_id]
    similarities = cosine_similarity_vec(user_profile, item_features)

    
    # Trier par similarité
    sorted_indices = np.argsort(similarities)[::-1]
    
    # Convertir les indices en video_ids
    recommendations = []
    video_id_list = list(video_id_to_index.keys())
    
    for idx in sorted_indices:
        if idx < len(video_id_list):
            video_id = video_id_list[idx]
            recommendations.append(video_id)
            if len(recommendations) >= n_recommendations:
                break
    
    return recommendations

## 🔸 Étapes du programme

1. **Chargement des fichiers** :
   - Le modèle sauvegardé (`.pkl`) contient les profils utilisateurs et les caractéristiques des vidéos.
   - Les interactions utilisateurs (avec leur `watch_ratio`) sont dans un fichier CSV.
   - Le réseau social (qui sont les amis de qui) est aussi chargé.

2. **Boucle sur chaque utilisateur** :
   - Si l'utilisateur a regardé des vidéos (watch_ratio > 0.5), on utilise cela comme "vérité terrain".
   - Si l'utilisateur **n'a pas de données** (cas cold-start), on utilise la moyenne des profils de ses amis pour faire des recommandations.
   - S’il n’a **aucun ami actif**, il est ignoré.

3. **Calcul des métriques** pour chaque utilisateur et pour plusieurs valeurs de `k` (1, 10, 20, 50, 100).

4. **Affichage final** : pour chaque `k`, le programme affiche la **moyenne de la précision, du rappel et du NDCG** sur tous les utilisateurs évalués.


In [15]:
def evaluate_all_users(model_path, test_data_path, social_network_path):
    import pickle
    import pandas as pd
    import numpy as np

    ks = [1, 10, 20, 50, 100]

    # Charger le modèle
    with open(model_path, 'rb') as f:
        model_data = pickle.load(f)

    # Charger les données de test et social network
    test_data = pd.read_csv(test_data_path)
    social_network = pd.read_csv(social_network_path)
    social_network['friend_list'] = social_network['friend_list'].apply(eval)  # convertit les listes en objets Python

    user_ids = test_data['user_id'].unique()
    user_profiles = model_data['user_profiles']

    # Métriques
    metrics = {k: {'precision': [], 'recall': [], 'ndcg': []} for k in ks}
    evaluated_users = 0

    for user_id in user_ids:
        recommendations = load_and_recommend(model_path, user_id, n_recommendations=max(ks))
        user_test_data = test_data[test_data['user_id'] == user_id]
        relevant_items = user_test_data[user_test_data['watch_ratio'] > 0.5]['video_id'].tolist()

        # Si utilisateur cold (pas d'items pertinents)
        if not relevant_items:
            # Tenter profil ami
            friends = social_network[social_network['user_id'] == user_id]['friend_list']
            if friends.empty:
                continue  # pas d'amis -> pas d'évaluation possible
            friends = friends.values[0]
            friend_profiles = [user_profiles[f] for f in friends if f in user_profiles]
            if not friend_profiles:
                continue  # amis sans profil
            avg_friend_profile = np.mean(friend_profiles, axis=0)
            # Recommandations basées sur profil ami
            recommendations = recommend_from_profile(avg_friend_profile, model_data, n_recommendations=max(ks))

        if not recommendations:
            continue

        relevant_items_with_ratings = dict(zip(user_test_data['video_id'].tolist(), user_test_data['watch_ratio'].tolist()))

        for k in ks:
            top_k_recs = recommendations[:k]
            precision = calculate_precision_at_k(top_k_recs, relevant_items, k)
            recall = calculate_recall_at_k(top_k_recs, relevant_items, k)
            ndcg = calculate_ndcg_at_k(top_k_recs, relevant_items_with_ratings, k)

            metrics[k]['precision'].append(precision)
            metrics[k]['recall'].append(recall)
            metrics[k]['ndcg'].append(ndcg)

        evaluated_users += 1

    if evaluated_users == 0:
        print("Aucun utilisateur valide pour l'évaluation.")
        return

    for k in ks:
        print(f"\n--- Résultats pour k = {k} ---")
        print(f"Precision@{k}: {sum(metrics[k]['precision']) / evaluated_users:.4f}")
        print(f"Recall@{k}:    {sum(metrics[k]['recall']) / evaluated_users:.4f}")
        print(f"NDCG@{k}:      {sum(metrics[k]['ndcg']) / evaluated_users:.4f}")


def recommend_from_profile(user_profile, model_data, n_recommendations=10):
    from sklearn.metrics.pairwise import cosine_similarity

    item_features = model_data['item_features']
    video_id_to_index = model_data['video_id_to_index']
    user_profile = user_profile.reshape(1, -1)

    similarities = cosine_similarity(user_profile, item_features).flatten()
    sorted_indices = np.argsort(similarities)[::-1]

    recommendations = []
    for idx in sorted_indices:
        vid = video_id_to_index.get(idx)
        if vid is not None:
            recommendations.append(vid)
            if len(recommendations) >= n_recommendations:
                break
    return recommendations

evaluate_all_users(
    model_path='models/content_recommender.pkl',
    test_data_path='KuaiRec 2.0/data/small_matrix.csv',
    social_network_path='KuaiRec 2.0/data/social_network.csv'
)


Calculating cosine similarity...
Calculating cosine similarity...
Calculating cosine similarity...
Calculating cosine similarity...
Calculating cosine similarity...
Calculating cosine similarity...
Calculating cosine similarity...
Calculating cosine similarity...
Calculating cosine similarity...
Calculating cosine similarity...
Calculating cosine similarity...
Calculating cosine similarity...
Calculating cosine similarity...
Calculating cosine similarity...
Calculating cosine similarity...
Calculating cosine similarity...
Calculating cosine similarity...
Calculating cosine similarity...
Calculating cosine similarity...
Calculating cosine similarity...
Calculating cosine similarity...
Calculating cosine similarity...
Calculating cosine similarity...
Calculating cosine similarity...
Calculating cosine similarity...
Calculating cosine similarity...
Calculating cosine similarity...
Calculating cosine similarity...
Calculating cosine similarity...
Calculating cosine similarity...
Calculatin

### 1. Precision@k
La précision est relativement élevée à tous les niveaux de *k* :

- Par exemple, **Precision@1 = 0.5379**, ce qui signifie que dans **53.79% des cas**, l’élément en première position est pertinent.
- Cependant, la précision diminue légèrement avec l’augmentation de *k*, ce qui est attendu 

---

### 2. Recall@k
Le **recall** est extrêmement faible :

- **Recall@1 = 0.0003**, **Recall@100 = 0.0188**.
- Cela signifie que le système retrouve **moins de 2% des éléments pertinents** dans les 100 recommandations.
- C’est un **signe alarmant** que le système **rate massivement** les items pertinents, **malgré une bonne précision apparente**.

---

### 3. NDCG@k
Le **NDCG (Normalized Discounted Cumulative Gain)** reste très bas :

- **NDCG@1 = 0.0086**, **NDCG@100 = 0.0257**.
- Cela indique que **l’ordre des recommandations ne reflète pas bien la pertinence des items**.
- Même les quelques items pertinents retrouvés sont **souvent mal classés**.


## Tentatives d'amélioration des performances

Malgré mes efforts et plusieurs tentatives pour améliorer les performances du système de recommandation — en particulier les métriques **NDCG** et **Recall** — je n'ai pas réussi à obtenir de résultats significativement meilleurs que ceux présentés.

### Ce que j'ai essayé :
- Ajustement du nombre de recommandations (`k`) pour observer l’impact sur les résultats.
- Utilisation du profil moyen des amis pour les utilisateurs "cold-start".
- Modification des hyperparametres.